In [22]:
import srt_util
import matplotlib.pyplot as plt
import numpy as np
import PhysicsTool as pt
import sympy
pt.plotting.set_theme()
np.set_printoptions(precision=3, suppress=False)

In [ ]:

with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp1_RefCold_T1225_20251002.hdf") as srt_reader:
    P_ref_cold1 = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp1_RefHot_T1225_20251002.hdf") as srt_reader:
    P_ref_hot1  = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp1_Sun_T1225_20251002.hdf") as srt_reader:
    P_sun1 = np.average(srt_reader.get_power_spectrum().sum(axis=1))

with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp2_RefCold_T1228_20251002.hdf") as srt_reader:
    P_ref_cold2 = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp2_RefHot_T1228_20251002.hdf") as srt_reader:
    P_ref_hot2  = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp2_Sun_T1228_20251002.hdf") as srt_reader:
    P_sun2 = np.average(srt_reader.get_power_spectrum().sum(axis=1))

with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp3_RefCold_T1230_20251002.hdf") as srt_reader:
    P_ref_cold3 = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp3_RefHot_T1230_20251002.hdf") as srt_reader:
    P_ref_hot3  = np.average(srt_reader.get_power_spectrum().sum(axis=1))
with srt_util.SRT_HDF_Reader("Measurements/CaibAndSunTemp/SunTemp3_Sun_T1230_20251002.hdf") as srt_reader:
    P_sun3 = np.average(srt_reader.get_power_spectrum().sum(axis=1))

# print(P_ref_cold1)
# print(P_ref_hot1)
# print(P_sun1)

# print(" ")
# print(P_ref_cold2)
# print(P_ref_hot2)
# print(P_sun2)

# print(" ")
# print(P_ref_cold3)
# print(P_ref_hot3)
# print(P_sun3)

# All three measurements have almost same values. Use only Measurement 1
# Assume 5% error for integrated power
P_ERR_REL = 0.05
P_c = pt.UErr(P_ref_cold1, P_ref_cold1*P_ERR_REL)
P_h = pt.UErr(P_ref_hot1, P_ref_hot1*P_ERR_REL)
P_s = pt.UErr(P_sun1, P_sun1*P_ERR_REL)

# Reference Temperatures (Assume 1% error for T_hot and T_cold)
T_HOT_VAL = 562.7
T_COLD_VAL = 2.7
T_ERR_REL = 0.01
T_h = pt.UErr(T_HOT_VAL, T_HOT_VAL*T_ERR_REL)
T_c = pt.UErr(T_COLD_VAL, T_COLD_VAL*T_ERR_REL)

display("P_ref_hot [a.u.] = ", P_h)
display("P_ref_cold [a.u.] = ", P_c)
display("P_sun [a.u.] = ", P_s)


4.8825197802819996e-05
0.00027296749682027594
0.0013554988861961034


'P_ref_hot [a.u.] = '

(2.7 ± 0.1) × 10^-4

'P_ref_cold [a.u.] = '

(4.9 ± 0.2) × 10^-5

'P_sun [a.u.] = '

(1.36 ± 0.07) × 10^-3

T_N= 119.28550157393464


In [27]:
# Calc Gain G and Noise Temperature T_N with errors
# ----------------------------------------------------------------------------------------

P_c_sym = sympy.symbols("P_c")
P_h_sym = sympy.symbols("P_h")
T_h_sym = sympy.symbols("T_h")
T_c_sym = sympy.symbols("T_c")
T_N_sym = sympy.symbols("T_N")
G_sym = sympy.symbols("G")

# G Calculation and Error propagation (Formulas)
with sympy.evaluate(False):
    G_expr_sym = (P_h_sym - P_c_sym) / (T_h_sym - T_c_sym)
G_eq_sym = sympy.Eq(G_sym, G_expr_sym)
display(G_eq_sym)

def log(eq : sympy.Eq):
    display(eq)
    print(sympy.latex(eq))

G_err_expr = pt.u_error_propagation_formula(G_expr_sym, values = [P_c_sym,P_h_sym,T_c_sym,T_h_sym], target_symbol=G_sym, logger = log)
display(G_err_expr)

# T_N Calculation and Error propagation (Formulas)
with sympy.evaluate(False):
    T_N_expr_sym = (T_h_sym*P_c_sym - T_c_sym*P_h_sym) / (P_h_sym - P_c_sym)
T_N_eq_sym = sympy.Eq(T_N_sym, T_N_expr_sym)
display(T_N_eq_sym)

def log(eq : sympy.Eq):
    display(eq)
    print(sympy.latex(eq))

T_N_err_expr = pt.u_error_propagation_formula(T_N_expr_sym, values = [P_c_sym,P_h_sym,T_c_sym,T_h_sym], target_symbol=T_N_sym, logger = log)
display(T_N_err_expr)

# G Calculation and Error propagation (Values)
G = pt.err.u_propagate_error(G_expr_sym, {P_c_sym:P_c, P_h_sym:P_h, T_c_sym:T_c, T_h_sym:T_h})
print("G [a.u./K] = ", G)

# T_N Calculation and Error propagation (Values)
T_N = pt.err.u_propagate_error(T_N_expr_sym, {P_c_sym:P_c, P_h_sym:P_h, T_c_sym:T_c, T_h_sym:T_h})
print("T_N [K] = ", T_N)

Eq(G, (-P_c + P_h)/(-T_c + T_h))

Eq(s_G**2, s_P_c**2*Derivative(G(T_h, P_c, P_h, T_c), P_c)**2 + s_P_h**2*Derivative(G(T_h, P_c, P_h, T_c), P_h)**2 + s_T_c**2*Derivative(G(T_h, P_c, P_h, T_c), T_c)**2 + s_T_h**2*Derivative(G(T_h, P_c, P_h, T_c), T_h)**2)

s_{G}^{2} = s_{P c}^{2} \left(\frac{\partial}{\partial P_{c}} G{\left(T_{h},P_{c},P_{h},T_{c} \right)}\right)^{2} + s_{P h}^{2} \left(\frac{\partial}{\partial P_{h}} G{\left(T_{h},P_{c},P_{h},T_{c} \right)}\right)^{2} + s_{T c}^{2} \left(\frac{\partial}{\partial T_{c}} G{\left(T_{h},P_{c},P_{h},T_{c} \right)}\right)^{2} + s_{T h}^{2} \left(\frac{\partial}{\partial T_{h}} G{\left(T_{h},P_{c},P_{h},T_{c} \right)}\right)^{2}


Eq(s_G**2, s_P_c**2*(-1/(-T_c + T_h))**2 + s_P_h**2*(1/(-T_c + T_h))**2 + s_T_c**2*((-P_c + P_h)/(-T_c + T_h)**2)**2 + s_T_h**2*(-(-P_c + P_h)/(-T_c + T_h)**2)**2)

s_{G}^{2} = s_{P c}^{2} \left(- \frac{1}{- T_{c} + T_{h}}\right)^{2} + s_{P h}^{2} \left(\frac{1}{- T_{c} + T_{h}}\right)^{2} + s_{T c}^{2} \left(\frac{- P_{c} + P_{h}}{\left(- T_{c} + T_{h}\right)^{2}}\right)^{2} + s_{T h}^{2} \left(- \frac{- P_{c} + P_{h}}{\left(- T_{c} + T_{h}\right)^{2}}\right)^{2}


...

s_{T N}^{2} = s_{P c}^{2} \left(\frac{T_{h}}{- P_{c} + P_{h}} + \frac{P_{c} T_{h} - P_{h} T_{c}}{\left(- P_{c} + P_{h}\right)^{2}}\right)^{2} + s_{P h}^{2} \left(- \frac{T_{c}}{- P_{c} + P_{h}} - \frac{P_{c} T_{h} - P_{h} T_{c}}{\left(- P_{c} + P_{h}\right)^{2}}\right)^{2} + s_{T c}^{2} \left(- \frac{P_{h}}{- P_{c} + P_{h}}\right)^{2} + s_{T h}^{2} \left(\frac{P_{c}}{- P_{c} + P_{h}}\right)^{2}


Eq(s_T_N, sqrt(s_P_c**2*(T_h/(-P_c + P_h) + (P_c*T_h - P_h*T_c)/(-P_c + P_h)**2)**2 + s_P_h**2*(-T_c/(-P_c + P_h) - (P_c*T_h - P_h*T_c)/(-P_c + P_h)**2)**2 + s_T_c**2*(-P_h/(-P_c + P_h))**2 + s_T_h**2*(P_c/(-P_c + P_h))**2))

s_{T N} = \sqrt{s_{P c}^{2} \left(\frac{T_{h}}{- P_{c} + P_{h}} + \frac{P_{c} T_{h} - P_{h} T_{c}}{\left(- P_{c} + P_{h}\right)^{2}}\right)^{2} + s_{P h}^{2} \left(- \frac{T_{c}}{- P_{c} + P_{h}} - \frac{P_{c} T_{h} - P_{h} T_{c}}{\left(- P_{c} + P_{h}\right)^{2}}\right)^{2} + s_{T c}^{2} \left(- \frac{P_{h}}{- P_{c} + P_{h}}\right)^{2} + s_{T h}^{2} \left(\frac{P_{c}}{- P_{c} + P_{h}}\right)^{2}}


sqrt(s_P_c**2*(T_h/(-P_c + P_h) + (P_c*T_h - P_h*T_c)/(-P_c + P_h)**2)**2 + s_P_h**2*(-T_c/(-P_c + P_h) - (P_c*T_h - P_h*T_c)/(-P_c + P_h)**2)**2 + s_T_c**2*(-P_h/(-P_c + P_h))**2 + s_T_h**2*(P_c/(-P_c + P_h))**2)

G [a.u./K] =  (4.0 ± 0.3) × 10^-7
T_N [K] =  119 ± 10


In [28]:
# Antenna Temperature, pointing to the sun
# ----------------------------------------------------------------------------------------
T_A_sym = sympy.symbols("T_A")
P_s_sym = sympy.symbols("P_s")

# T_A Calculation and Error propagation (Formulas)
with sympy.evaluate(False):
    T_A_expr_sym = P_s_sym / G_sym - T_N_sym
T_A_eq_sym = sympy.Eq(T_A_sym, T_A_expr_sym)
display(T_A_eq_sym)

def log(eq : sympy.Eq):
    display(eq)
    print(sympy.latex(eq))

T_A_err_expr = pt.u_error_propagation_formula(T_A_expr_sym, values = [P_s_sym,G_sym,T_N_sym], target_symbol=T_A_sym, logger = log)
display(T_A_err_expr)

# T_A Calculation and Error propagation (Values)
T_A = pt.err.u_propagate_error(T_A_expr_sym, {P_s_sym:P_s, G_sym:G, T_N_sym:T_N})
print("T_A [K] = ", T_A)


Eq(T_A, -T_N + P_s/G)

Eq(s_T_A**2, s_G**2*Derivative(T_A(P_s, G, T_N), G)**2 + s_P_s**2*Derivative(T_A(P_s, G, T_N), P_s)**2 + s_T_N**2*Derivative(T_A(P_s, G, T_N), T_N)**2)

s_{T A}^{2} = s_{G}^{2} \left(\frac{\partial}{\partial G} T_{A}{\left(P_{s},G,T_{N} \right)}\right)^{2} + s_{P s}^{2} \left(\frac{\partial}{\partial P_{s}} T_{A}{\left(P_{s},G,T_{N} \right)}\right)^{2} + s_{T N}^{2} \left(\frac{\partial}{\partial T_{N}} T_{A}{\left(P_{s},G,T_{N} \right)}\right)^{2}


Eq(s_T_A**2, s_P_s**2*(1/G)**2 + s_G**2*(-P_s/G**2)**2 + s_T_N**2*(-1)**2)

s_{T A}^{2} = s_{P s}^{2} \left(\frac{1}{G}\right)^{2} + s_{G}^{2} \left(- \frac{P_{s}}{G^{2}}\right)^{2} + \left(-1\right)^{2} s_{T N}^{2}


Eq(s_T_A, sqrt(s_P_s**2*(1/G)**2 + s_G**2*(-P_s/G**2)**2 + s_T_N**2*(-1)**2))

s_{T A} = \sqrt{s_{P s}^{2} \left(\frac{1}{G}\right)^{2} + s_{G}^{2} \left(- \frac{P_{s}}{G^{2}}\right)^{2} + \left(-1\right)^{2} s_{T N}^{2}}


sqrt(s_P_s**2*(1/G)**2 + s_G**2*(-P_s/G**2)**2 + s_T_N**2*(-1)**2)

T_A [K] =  (3.3 ± 0.3) × 10^3
